In [27]:
import matplotlib.colors as mcolors
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from interpreto.attributions.methods import OcclusionExplainer
from interpreto.commons.granularity import GranularityLevel
from interpreto.visualizations.attributions.classification_highlight import (
    MultiClassAttributionVisualization,
    SingleClassAttributionVisualization,
)

In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

explainer = OcclusionExplainer(model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.WORD)

In [36]:
attribution_outputs = explainer.explain(
    model_inputs=[
        "I love programming in Python.",
        "The weather is terrible, today.",
        "This is a great movie!",
        "I am not a fan of this book.",
    ]
)

In [37]:
attribution_outputs

[AttributionOutput(attributions=tensor([[0.5254, 5.2755, 1.4885, 0.3053, 1.0455, 2.4529]],
        grad_fn=<SqueezeBackward1>), elements=['i', 'love', 'programming', 'in', 'python', '.']),
 AttributionOutput(attributions=tensor([[-0.0217, -0.3115,  0.0180,  1.2174,  0.1757, -0.1634,  0.1834]],
        grad_fn=<SqueezeBackward1>), elements=['the', 'weather', 'is', 'terrible', ',', 'today', '.']),
 AttributionOutput(attributions=tensor([[-0.0278,  0.3640,  0.1230,  4.3206,  2.0256,  1.5795]],
        grad_fn=<SqueezeBackward1>), elements=['this', 'is', 'a', 'great', 'movie', '!']),
 AttributionOutput(attributions=tensor([[-0.3165, -0.1042,  4.4913,  0.2782, -1.0968, -0.6034, -0.0196,  0.2770,
           0.4905]], grad_fn=<SqueezeBackward1>), elements=['i', 'am', 'not', 'a', 'fan', 'of', 'this', 'book', '.'])]

In [ ]:
tokenizer_name = [
    "/data/models/notams/distilbert_tokenizer_no-preproc",
    "/data/models/notams/distilbert_tokenizer_preproc",
]
model_name = [
    "/data/models/notams/distillbert_base_trained-6ep_01-2025",
    "/data/models/notams/distillbert_base_trained-8ep_no-preproc_base-token_03-2025",
    "/data/models/notams/distillbert_base_trained-8ep_preproc_base-token_03-2025",
    "/data/models/notams/distillbert_base_trained-8ep_preproc_token-trained_03-2025",
]

model = AutoModelForSequenceClassification.from_pretrained(model_name[0], num_labels=13)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name[1])

explainer = OcclusionExplainer(model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.WORD)

In [46]:
tokenizer("INCREASED MIL ACFT (PC21) ACT IN VCY AD CONSISTING OF FORMATION AND AEROBATIC MANEUVERING")

{'input_ids': [2, 808, 453, 210, 1, 1, 1, 228, 117, 939, 160, 8525, 120, 4385, 166, 103, 3735, 1686, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [44]:
attribution_outputs = explainer.explain(
    model_inputs=[
        "ILS-LOC,GP,DME,IM FOR RWY 06R-U/S DUE TO FLTCK",
        "/FNP/ LOC TYPE DIRECTIONAL AID RWY 28R LOC/DME/GP U/S.",
        "INCREASED MIL ACFT (PC21) ACT IN VCY AD CONSISTING OF FORMATION AND AEROBATIC MANEUVERING",
    ]
)

In [45]:
attribution_outputs

[AttributionOutput(attributions=tensor([[-0.4121, -0.1906, -0.2922, -0.1754, -0.2601, -0.0632, -0.1618, -0.1431,
          -0.1512, -0.1214, -0.0854, -0.0463, -0.1070, -0.1027, -0.0538, -0.1366,
          -0.2785,  1.4600, -0.3176]], grad_fn=<SqueezeBackward1>), elements=['ils', '-', 'loc', '', 'gp', '', 'dme', '', 'im', 'for', 'rwy', '', '-', 'u', '', 's', 'due', 'to', 'fltck']),
 AttributionOutput(attributions=tensor([[ 0.7749,  2.0595,  0.8485, -0.0067,  0.0816,  2.5420, -0.6612,  0.2265,
           0.1996,  0.0886,  0.1089,  0.0846,  0.1458,  0.1635,  0.2483,  0.1298,
           0.2093,  0.2159]], grad_fn=<SqueezeBackward1>), elements=['', 'fnp', '', 'loc', 'type', 'directional', 'aid', 'rwy', '', 'loc', '', 'dme', '', 'gp', 'u', '', 's', '.']),
 AttributionOutput(attributions=tensor([[ 0.2793,  0.1534,  0.2543,  0.5795,  0.5365,  0.3773,  0.2129,  0.0336,
          -0.2988,  0.1687, -0.0741,  0.1755,  0.2014, -0.6746,  0.0806, -0.5863]],
        grad_fn=<SqueezeBackward1>), elemen

In [42]:
viz = SingleClassAttributionVisualization(
    attribution_output_list=attribution_outputs,
    color=mcolors.to_rgb("red"),
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

In [25]:
attribution_outputs_multipleclasses = explainer.explain(
    model_inputs=[
        "ILS-LOC,GP,DME,IM FOR RWY 06R-U/S DUE TO FLTCK",
        "/FNP/ LOC TYPE DIRECTIONAL AID RWY 28R LOC/DME/GP U/S.",
        "INCREASED MIL ACFT (PC21) ACT IN VCY AD CONSISTING OF FORMATION AND AEROBATIC MANEUVERING",
    ],
    targets=torch.tensor([1, 2, 3]),
)

In [26]:
viz = MultiClassAttributionVisualization(
    attribution_output_list=attribution_outputs_multipleclasses,
    class_colors=[mcolors.to_rgb("green"), mcolors.to_rgb("blue"), mcolors.to_rgb("orange")],
    class_names=["class1", "class2", "class 3"],
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()